In [1]:
# default_exp transfer_dtsek_to_dergey

In [50]:
import yaml
import unicodedata

from antx.utils import optimized_diff_match_patch
from openpecha.serializers import Serialize
from tqdm import tqdm

from config import config

In [26]:
dmp = optimized_diff_match_patch()
dmp.binary_path

PosixPath('/home/tenzin/.antx/bin/dmp')

# Transfer Annotations

Steps:
1. Get dergey base for text
1. Get pedurma double tseked text
1. Transfer pedurma double tseks to dergey base

## Get dergey base for text

In [8]:
#export
def get_index_layer(path):
    index_layer = yaml.safe_load((path/'index.yml').open())
    return index_layer

def get_base_text(text_id, opf_path, index_layer):
    serializer = Serialize(opf_path, text_id=text_id, index_layer=index_layer)
    #base_text = ''.join(serializer.get_text_base_layer().values())
    base_text = list(serializer.get_text_base_layer().values())[0]
    return base_text

In [9]:
# base_text = get_base_text('D1109', config.tengyur_opf, index_layer)
# base_text[:2000]

## Get pedurma double tseked text

## Transfer pedurma double tseks to dergey base

In [10]:
#export
def isNSM(char):
    # Detects nonspacing mark characters
    if unicodedata.category(char) == "Mn":
        return True
    return False

In [11]:
char = 'ཽ'
isNSM(char)

True

In [12]:
#export
def get_first_char_idx(text, char):
    """Return first char idx in `text` and -1 of not found

    found har idx is expanded for whitespces after the char.
    """
    is_char_found = False
    idx = -1
    for i in range(len(text)):
        if not is_char_found and text[i] == char:
            is_char_found = True
            idx = i
        elif is_char_found:
            if text[i] != ' ': return i-1
            elif i == len(text)-1: return i
    return idx

In [13]:
string = '01|  567 9'
idx = get_first_char_idx(string, '|')
print(repr(string[:idx+1]), repr(string[idx+1]))

string = '01|  '
idx = get_first_char_idx(string, '|')
print(repr(string[:idx+1]))

'01|  ' '5'
'01|  '


In [43]:
#export
def parse_double_tsek(text):
    double_tsek_idxs = []
    base_char_idx = 0
    for c in text:
        if c == config.double_tsek_sym:
            double_tsek_idxs.append(base_char_idx)
            continue
        base_char_idx += 1
    return double_tsek_idxs

def adjust_next_diff(i, diffs, ann_text, to_char):
    diff_mode, diff_chunk = diffs[i+1]
    # add chars till next tsek to the ann_text
    first_char_idx = get_first_char_idx(diff_chunk, to_char)
    ann_text += diff_chunk[:first_char_idx+1]
    # remove chars till next tsek from diff[i+1]
    diffs[i+1] = (diff_mode, diff_chunk[first_char_idx+1:])
    return ann_text


def transfer_anns_with_diff(base_text, dest_text):
    ann_text = ''
    diffs = list(dmp.diff_main(dest_text, base_text))
    for i in range(len(diffs)):
        if diffs[i][0] == -1:
            if config.double_tsek_sym in diffs[i][1]:
                # check for next diff adjustment
                if i < len(diffs)-1:
                    # adjust next diff[i+1] if it's first char is NSM
                    if isNSM(diffs[i+1][1][0]):
                        ann_text = adjust_next_diff(i, diffs, ann_text, config.tsek)

                    # adjust next diff[i+1] if it's first char in tsek
                    elif diffs[i+1][1][0] == config.tsek:
                        ann_text = adjust_next_diff(i, diffs, ann_text, config.tsek)

                    # adjust next diff[i+1] if it's first char in shed
                    elif diffs[i+1][1][0] == config.shed:
                        ann_text = adjust_next_diff(i, diffs, ann_text, config.shed)

                    # adjust next diff[i+1] if its first char is line return
                    elif diffs[i+1][1][0] == '\n':
                        ann_text = adjust_next_diff(i, diffs, ann_text, '\n')

                if len(diffs[i][1]) == 1:
                    ann_text += diffs[i][1]
                else:
                    ann_text += '$'
        else:
            ann_text += diffs[i][1]
    double_tsek_idxs = parse_double_tsek(ann_text)
    print(f'\t- Transferred {len(double_tsek_idxs)} out of {dest_text.count(config.double_tsek_sym)}')
    return double_tsek_idxs, ann_text

In [44]:
#NSM case
p_text = 'དེ་ནས་ཀོ$ཧཱུཾ་བཱི་ན་'
d_text = 'དེ་ནས་ཀཽ་ཤཱཾ་བཱི་ན་གནས་'
transfer_anns_with_diff(d_text, p_text)

	- Transferred 1 out of 1


([9], 'དེ་ནས་ཀཽ་$ཤཱཾ་བཱི་ན་གནས་')

In [45]:
# adjust marker which is diffed before tsek
p_text = 'རིག་པས$༔མ་རིག་པའི་སྒོ་'
d_text = 'རིག་པས་མ་རིག་པའི་སྒོ་'
transfer_anns_with_diff(d_text, p_text)

	- Transferred 1 out of 1


([7], 'རིག་པས་$མ་རིག་པའི་སྒོ་')

In [46]:
p_text = 'ནི༑$ལྷ་སྦྱིན་འདི་ཉིད་'
d_text = '་ནི། ལྷ་སྦྱིན་འདི་ཉིད་'
transfer_anns_with_diff(d_text, p_text)

	- Transferred 1 out of 1


([5], '་ནི། $ལྷ་སྦྱིན་འདི་ཉིད་')

In [47]:
p_text = 'ཟད་དེ། $ད་ནི་དེ་'
d_text = 'ཟད་དེ།\nད་ནི་དེ་'
transfer_anns_with_diff(d_text, p_text)

	- Transferred 1 out of 1


([7], 'ཟད་དེ།\n$ད་ནི་དེ་')

In [4]:
pedurma_ann_text = 'ལས་བརྒྱ་ཐམ་པ་པ། ༄༅། །རྒྱ་གར་སྐད་དུ། ཀརྨ་ཤ་ཏ་ཀ། བོད་སྐད་དུ། ལས་བརྒྱ་ཐམ་པ་པོ། བམ་པོ་དང་པོ། ཐམས་ཅད་མཁྱེན་པ་ལ་ཕྱག་འཚལ་ལོ། །གང་ལས་འཇིག་རྟེན་བླ་མ་བདེ་གཤེགས་ཐོས་པའི་སྒོ་ནས་རབ་སྙན་བརྟན་པའི་གསུང་་་་ལྡན་གྱིས། །སེམས་ཅན་རྣམས་ལ་ཕན་པ་འབའ་ཞིག་བཞེད་ཕྱིར་བཤད་པ་རྣམ་པ་སྣ་ཚོགས་རང་ཉིད་ཀྱིས། །ལོག་པར་ལྟ་བའི་མུན་ནག་ཆེན་པོ་ཐིབས་པོར་$འཐོམས་ཤིངའཁྲུགས་པ་རྣམས་ལ་རབ་གསུངས་པ། །$དེ་ཡི་མིང་ནི་ལས་རྣམ་བརྒྱ་པ་ཞེས་བྱ་ཡོངས་སུ་ཚང་བ་བདག་གིས་བཤད་ཀྱིས་ཉོན། །སྤྱི་སྡོམ་ནི༑ཁྱི་མོ་དང་ནི་ཤིང་རྟ་དང་། །ཀ་ཙང་ཀ་ལ་བྱམས་མི་སྡུག། བྱ་དང་འཕྱེ་བོ་གང་པོ་དང་། །བུ་རྣམས་དང་ནི་བརྒྱ་བྱིན་ནོ། །སྡོམ་ནི།'
derge_text = '༄༅༅། །རྒྱ་གར་སྐད་དུ། ཀརྨ་ཤ་ཏ་ཀ། བོད་སྐད་དུ། ལས་བརྒྱ་ཐམ་པ་པ། བམ་པོ་དང་པོ། ཐམས་ཅད་མཁྱེན་པ་ལ་ཕྱག་འཚལ་ལོ། །གང་ལས་འཇིག་རྟེན་བླ་མ་བདེ་གཤེགས་ཐོས་པའི་སྒོ་ནས་རབ་སྙན་བརྟན་པའི་གསུང་ལྡན་གྱིས། །སེམས་ཅན་རྣམས་ལ་ཕན་པ་འབའ་ཞིག་བཞེད་ཕྱིར་བཤད་པ་རྣམ་པ་སྣ་ཚོགས་རང་ཉིད་ཀྱིས། །ལོག་པར་ལྟ་བའི་མུན་ནག་ཆེན་པོ་ཐིབས་པོར་འཐོམས་ཤིང་འཁྲུགས་པ་རྣམས་ལ་རབ་གསུངས་པ། །དེ་ཡི་མིང་ནི་ལས་རྣམ་བརྒྱ་པ་ཞེས་བྱ་ཡོངས་སུ་ཚང་བ་བདག་གིས་བཤད་ཀྱིས་ཉོན། །སྤྱི་སྡོམ་ནི། ཁྱི་མོ་དང་ནི་ཤིང་རྟ་དང་། །ཀ་ཙང་ཀ་ལ་བྱམས་མི་སྡུག །བྱ་དང་འཕྱེ་བོ་གང་པོ་དང་། །བུ་རྣམས་དང་ནི་བརྒྱ་བྱིན་ནོ། །སྡོམ་ནི། ཁྱི་མོ་མིག་ཆུང་'
transfer_anns_with_diff(derge_text, pedurma_ann_text)

NameError: name 'transfer_anns_with_diff' is not defined

In [1]:
from config import TengyurConfig

In [2]:
config = TengyurConfig()
vars(config)

{'ocr_path': PosixPath('archive'),
 'peydurma_path': PosixPath('data/peydurma'),
 'template_path': PosixPath('data/peydurma/templates'),
 'output_path': PosixPath('dtsek/output'),
 'output_tmp_path': PosixPath('dtsek/output/tmp'),
 'double_tsek_sym': '$',
 'expected_double_tsek_sym': ':',
 'tsek': '་',
 'shed': '།',
 'img_size': (3969, 2641),
 'debug': False,
 'name': 'Tengyur',
 'work_id': 'W1PD95844',
 'd_pecha_id': 'P000002',
 'p_pecha_id': 'P000792',
 'images_path': PosixPath('archive/images/W1PD95844'),
 'ocr_output_path': PosixPath('archive/output/W1PD95844')}